<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4: West Nile Virus Prediction

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from pycaret.classification import *

In [3]:
train = pd.read_csv('../datasets/train_clean.csv')
test= pd.read_csv('../datasets/test_clean.csv')

In [4]:
train.head()

,latitude,longitude,wnvpresent,week,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity
0,41.867108,-87.654224,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
1,41.867108,-87.654224,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
2,41.862292,-87.648860,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
3,41.896282,-87.655232,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
4,41.907645,-87.760886,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249


In [5]:
test.head()

,id,latitude,longitude,week,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity
0,1,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
1,2,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
2,3,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
3,4,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
4,5,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192


## Baseline Model

## Pycaret

### Setting Up Environment

First we use the `setup()` function to initialize the environment in pycaret and create the transformation pipeline to prepare the data for modeling and deployment. It takes two mandatory parameters: our train dataset and target column (our y variable) which is `'wnvpresent'`. 

When `setup()` is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties and display a table containing the features and their inferred data types after setup() is executed.

Our train and test datasets have gone through `OneHotEncoding` for the categorical features in the 1st notebook. Pycaret is able to scale our data using the `normalize=True` parameter and we will be using z-score which is the same as `StandardScaler` (or Z-score normalization). Continuous features like `'relativehumidity'` has a wide range and due to differences in magnitude of features, our model may neglect units of higher number. 

Pycaret is able to detect and classify the features of the dataset. However, the features that `OneHotEncoding` had transformed was detected to be of categorical in nature instead. To bypass this, we had to list out these features to override the classification before running the dataset through Pycaret environment.


In [6]:
pce_1 = setup(data = train, target = 'wnvpresent', session_id = 5, 
              data_split_stratify=True, 
              normalize=True, normalize_method='zscore', #to scale our data
              numeric_features = ['week', 'month_5', 'month_6','month_7','month_8','month_9','month_10',
                                  'species_culex pipiens','species_culex pipiens/restuans','species_culex restuans','species_others'
                                 ] #since the categorical features has been converted, it ought to be treated as numerical features
             )

,Description,Value
0,session_id,5
1,Target,wnvpresent
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(8475, 25)"
5,Missing Values,False
6,Numeric Features,24
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
%time compare_models(n_select=2, sort = 'AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9472,0.8438,0.0719,0.6700,0.1225,0.1131,0.1909,0.2220
lightgbm,Light Gradient Boosting Machine,0.9447,0.8406,0.1438,0.4641,0.2149,0.1949,0.2332,0.0500
ada,Ada Boost Classifier,0.9461,0.8328,0.0344,0.3817,0.0607,0.0547,0.0976,0.0840
xgboost,Extreme Gradient Boosting,0.9390,0.8301,0.1656,0.3484,0.2204,0.1939,0.2098,0.3260
lr,Logistic Regression,0.9461,0.8038,0.0000,0.0000,0.0000,0.0000,0.0000,0.6710
lda,Linear Discriminant Analysis,0.9461,0.7971,0.0000,0.0000,0.0000,0.0000,0.0000,0.0130
rf,Random Forest Classifier,0.9396,0.7890,0.0969,0.2719,0.1371,0.1162,0.1338,0.1780
nb,Naive Bayes,0.2787,0.7816,0.9906,0.0690,0.1291,0.0314,0.1235,0.0080
et,Extra Trees Classifier,0.9331,0.7504,0.1250,0.2445,0.1620,0.1324,0.1413,0.1500
knn,K Neighbors Classifier,0.9408,0.7061,0.0469,0.2679,0.0773,0.0617,0.0889,0.0820


CPU times: user 6.02 s, sys: 356 ms, total: 6.38 s
Wall time: 22.4 s


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=5, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, 

Once the environment was setup, we were able to generate models based on default parameters in the Pycaret library. The `compare_models` function trains all models in the model library and scores them using stratified cross validation for metric evaluation. The output prints a score grid that shows average Accuracy, AUC, Recall, Precision, F1, Kappa, and MCC across the folds (10 by default) along with training times. The score grid with the highest performing metrics are highlighted accordingly. The grid by default is sorted using 'AUC' (highest to lowest) as the Kaggle competition will be looking at ROC-AUC scores for the submission. Therefore, we will be looking to optimize AUC as much as possible, while maintaining a good score in Accuracy and Precision.

The `compare_models` also returned the best performing models based on AUC scores with the `n_select=2` parameter which gives us the top 2 highest performing models which are Gradient Boosting Classifier and Light Gradient Boosting Machine Classifier. After narrowing down to these two models, we will attempt to increase its performance by tuning its hyperparameters. Pycaret also has the function to optimize the models generated by using the `tune_model` that helps tune the hyperparameters of the models. As mentioned earlier, we will be focusing on AUC score and the function will focus in optimizing based on AUC score by passing in the `optimize='AUC'` parameter.

### Light Gradient Boosting Model

In [8]:
#create lgbm model based on default param
lgbm_model = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9495,0.8453,0.2500,0.5714,0.3478,0.3257,0.3562
1,0.9495,0.8608,0.1562,0.6250,0.2500,0.2335,0.2956
2,0.9427,0.8241,0.1250,0.4000,0.1905,0.1691,0.2006
3,0.9477,0.8515,0.1250,0.5714,0.2051,0.1894,0.2503
4,0.9477,0.8815,0.2812,0.5294,0.3673,0.3427,0.3615
5,0.9494,0.8744,0.1562,0.6250,0.2500,0.2335,0.2955
6,0.9444,0.8170,0.0625,0.4000,0.1081,0.0949,0.1412
7,0.9376,0.8250,0.0938,0.2727,0.1395,0.1151,0.1331
8,0.9477,0.8133,0.1562,0.5556,0.2439,0.2256,0.2756


In [9]:
#test scores based on default param
pred_lgbm = predict_model(lgbm_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9410,0.8642,0.1241,0.3617,0.1848,0.1617,0.1871


In [10]:
#tuning lgbm model
%time tuned_lgbm = tune_model(lgbm_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9478,0.8623,0.2188,0.5385,0.3111,0.2890,0.3211
1,0.9512,0.8797,0.1250,0.8000,0.2162,0.2046,0.3045
2,0.9477,0.8471,0.0938,0.6000,0.1622,0.1498,0.2229
3,0.9460,0.8649,0.0938,0.5000,0.1579,0.1433,0.1996
4,0.9410,0.8592,0.2188,0.4118,0.2857,0.2579,0.2721
5,0.9511,0.8823,0.1250,0.8000,0.2162,0.2046,0.3045
6,0.9477,0.8347,0.0312,1.0000,0.0606,0.0575,0.1721
7,0.9376,0.8260,0.0938,0.2727,0.1395,0.1151,0.1331
8,0.9494,0.8357,0.0938,0.7500,0.1667,0.1566,0.2539


CPU times: user 3.91 s, sys: 210 ms, total: 4.12 s
Wall time: 20.9 s


In [11]:
#showing the best params
print(tuned_lgbm)

LGBMClassifier(bagging_fraction=0.7, bagging_freq=4, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.9,
               n_estimators=240, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5, reg_alpha=0.01, reg_lambda=1e-06, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [12]:
#generate interface that will display various plots
evaluate_model(tuned_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [13]:
#test scores based on tuned model
pred_holdout_lgbm = predict_model(tuned_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9426,0.8617,0.0949,0.3714,0.1512,0.1321,0.1662


We analyze the performance of the Light Gradient Boosting Machine (LGBM) model by using the `evaluate_model()` function which displays a user interface for all of the available plots for a given model. It internally uses the plot_model() function. The main focus in this portion is to evaluate the model based on the ROC-AUC score and graph. With an AUC score of 0.86, it suggests the 'west nile virus' and 'no west nile virus' populations are well separated, and the model is nearly as good as it can get. It also shows that the model is better at predicting 'no west nile virus' populations at at lower decision threshold, with the ROC for class 0 being higher and steeper up to 0.2 False Positive Rate.

The scores generated after running the model through the holdout (or 'test' set after Pycaret internal train-test split) also showed good predictive power with Accuracy and AUC being 0.94 and 0.86 respectively. From the markdown table below, we see the improvement in the model after the hyperparameter tuning as well:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC |
|---|---|---|---|---|
| Baseline (default)  | - | - | - | - |
| LGBM (default) | 0.9447 | 0.9410 | 0.8406 | 0.8642 |
| LGBM  (tuned) | 0.9459 | 0.9426 | 0.8617 | 0.8642 |

### Gradient Boosting Model

In [14]:
#create gbc model based on default params
%time gbc_model = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9529,0.8509,0.2188,0.7000,0.3333,0.3158,0.3745
1,0.9495,0.8764,0.0625,1.0000,0.1176,0.1120,0.2436
2,0.9477,0.8561,0.0312,1.0000,0.0606,0.0575,0.1721
3,0.9477,0.8485,0.0312,1.0000,0.0606,0.0575,0.1721
4,0.9528,0.8612,0.1562,0.8333,0.2632,0.2504,0.3487
5,0.9460,0.8576,0.0312,0.5000,0.0588,0.0528,0.1148
6,0.9477,0.8444,0.0625,0.6667,0.1143,0.1060,0.1934
7,0.9427,0.8286,0.0625,0.3333,0.1053,0.0898,0.1250
8,0.9460,0.8278,0.0312,0.5000,0.0588,0.0528,0.1148


CPU times: user 1.33 s, sys: 41.4 ms, total: 1.37 s
Wall time: 3.33 s


In [15]:
#test scores based on default param
pred_gbc = predict_model(gbc_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.9477,0.8494,0.0584,0.6667,0.1074,0.0996,0.1869


In [16]:
%time tuned_gbc = tune_model(gbc_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9478,0.8559,0.1875,0.5455,0.2791,0.2586,0.2991
1,0.9478,0.8622,0.0938,0.6000,0.1622,0.1498,0.2229
2,0.9477,0.8534,0.0625,0.6667,0.1143,0.1060,0.1934
3,0.9477,0.8597,0.0312,1.0000,0.0606,0.0575,0.1721
4,0.9477,0.8643,0.2188,0.5385,0.3111,0.2889,0.3210
5,0.9494,0.8730,0.0625,1.0000,0.1176,0.1120,0.2436
6,0.9477,0.8409,0.0312,1.0000,0.0606,0.0575,0.1721
7,0.9410,0.8318,0.0938,0.3333,0.1463,0.1256,0.1535
8,0.9444,0.8348,0.0938,0.4286,0.1538,0.1371,0.1812


CPU times: user 2.88 s, sys: 189 ms, total: 3.06 s
Wall time: 1min 28s


In [17]:
#showing the best params
print(tuned_gbc)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=2,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=2,
                           min_samples_split=10, min_weight_fraction_leaf=0.0,
                           n_estimators=270, n_iter_no_change=None,
                           presort='deprecated', random_state=5, subsample=1.0,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [18]:
#generate interface that will display various plots
evaluate_model(tuned_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [19]:
#test scores based on tuned model
pred_holdout_gbc = predict_model(tuned_gbc)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.9457,0.8706,0.0803,0.4783,0.1375,0.1239,0.1796


We did the same steps as when generating the model for LGBM for Gradient Boosting Classifier (GBC). After optimizing the model, the scores showed an improvement as well. From the ROC-AUC graph, we can see it has a similar score to LGBM but it does show an improvement by 0.01 with AUC score of 0.87. Again, this model also shows that it is better at predicting 'no west nile virus' populations at at lower decision threshold, with the ROC for class 0 being higher and steeper up to 0.2 False Positive Rate.

The scores generated after running the model through the holdout set also showed good predictive power with Accuracy and AUC being 0.94 and 0.87 respectively. Below, we are able to see the difference in scores before hyperparameter tuning and evaluate which models showed the best potential:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC |
|---|---|---|---|---|
| Baseline (default)  | - | - | - | - |
| LGBM (default) | 0.9447 | 0.9410 | 0.8406 | 0.8642 |
| LGBM  (tuned) | 0.9459 | 0.9426 | 0.8617 | 0.8642 |
| GBC (default) | 0.9477 | 0.9477 | 0.8438 | 0.8494 |
| GBC (tuned) | 0.9454 | 0.9457 | 0.8490 | 0.8706 |

Both LGBM and GBC outperformed out baseline models and after comparing the two models, it seems like GBC has the best scores when predicting the holdout set in Pycaret, even though LGBM had slighly better scores in the training. Thus, we will be using Gradient Boosting Classifier as our final model to predict the test dataset and submit it to Kaggle.


In [20]:
#finalizing the model by training it on the complete dataset before it is deployed for predicting the test datasest
final_gbc = finalize_model(tuned_gbc)

In [21]:
#predicting on test dataset
pred_unseen_gbc = predict_model(final_gbc, data = test)

## Kaggle Submission

In [22]:
#making function to change the score and making appropriate dataframe for Kaggle submission
def kaggle_submission(preds, model_name):
    
    #changing the scores
    preds['Prob'] = preds['Score']
    preds.loc[preds['Label'] == 0, 'Prob'] = 1-preds['Prob']
    
    #making appropriate dataframe for submission
    submission = pd.merge(test['id'], preds['Prob'], left_index = True, right_index = True)
    
    submission.rename({'id' : 'Id',
                      'Prob' : 'WnvPresent'},
                     inplace = True,
                     axis = 1)
    
    #export
    submission.to_csv(f'../datasets/submission_{model_name}.csv', index=False)
    
    return submission

In [23]:
kaggle_submission(pred_unseen_gbc, 'gbc')

,Id,WnvPresent
0,1,0.0090
1,2,0.0040
2,3,0.0069
3,4,0.0002
4,5,0.0002
...,...,...
116288,116289,0.0001
116289,116290,0.0001
116290,116291,0.0001
116291,116292,0.0001


Once we decided to use GBC as our final model, Pycaret requires us to use the `finalize_model` function. What this does is that it will train the final model by fitting it onto the complete dataset including the test/hold-out sample (30% in this case) that was used in the setup of Pycaret environment. The purpose of this function is to train the model on the complete dataset before it is deployed in production. Following which, the model is fed the unseen test dataset for the Kaggle submission.

The output of the `predict_model` will give us two new columns; `Label` and `Score`. 
* `Label` column will show the predicted classification (0 being absence of WNV and 1 being presence of WNV) by the model.
* `Score` column will show the probability that the Label is of the positive class.  

Therefore an additional step is required before making the submission to Kaggle which is to inverse the Score by subracting 1 to get the probability that the Label is of the negative class. **This is only done for rows that are predicted to be 0 in the Label column.** Following which, a new dataframe was created to show only `Id` and `WnvPresent` for Kaggle scoring and submission.

After our submission to Kaggle with the predictions from the tuned GBC model, the final public score was **0.76133**.

## Cost-Benefit Analysis of Spraying

To determine the cost effectiveness of spraying, we first need to consider the following:  
* Direct costs: These would mainly include the cost of procuring chemicals required for spraying.  
* Indirect costs: These would be productivity/economic costs incurred from people seeking short/long term treatment for the West Nile Virus.

## Conclusion

The final model we built was successful in answering our problem statement (predicting the West Nile Virus (WNV) in mosquitoes across the city of Chicago for the missing years 2008, 2010, 2012 and 2014). With a ROC-AUC score of 0.76 from Kaggle based on unseen data and an accuracy score of 0.95 and AUC score of 0.87 based on test sets, our model is sufficient in its prediction. Thus, we are able to generally predict areas/clusters with potential WNV presence and with that information, can introduce potential preventive measures to cull the mosquito population and the WNV in Chicago.

Unfortunately, this model does not directly achieve the bigger goal of eradicating the West Nile Virus from Chicago. Most of the features in the model are beyond our control (such as weather), so inference would not directly help in suggesting ways to eradicate the virus. Furthermore, our model is only effective under certain weather conditions (for example hailstorms and fogs) might throw the predictions off. As mentioned, we have no control over the weather and due to climate change, certain weather patterns that may be seen in some years will not be seen in the next. Nevertheless, there are some aspects from our project that may assist in reducing the numbers to the brink of eliminating the virus in the future. Our findings have shown that the effect of spraying has a significant impact on reducing the number of mosquito clusters and WNV since its introduction in 2011 and we do have control over this, unlike the weather.

### Recommendations

Based on our project, there are measures within our control that will assist in reducing the number of mosquitoes. 

**Controlling the Spraying of Insecticides**  
As mentioned earlier, spraying did help to reduce clusters, especially in residential areas (with most clusters found around the edge of the area). Instead of spraying in areas without proper methods, we can focus on these three factors to improve it in terms of both cost effectiveness and efficacy:

1) Location  
Spraying can be targeted at locations where our model predicts to have a high probability of the occurrence of the virus.
2) Time of year  
Spraying efforts can be ramped up at the appropriate time of the year (July to August where higher number of mosquitoes are found) to reduce mosquito population.
3) Spray and wind direction  
Research shows that spraying along and against the wind will incur different efficacy. However, this needs further research as this can be a topic on its own.

The CDC has already implemented local [mosquito control programs](https://www.cdc.gov/mosquitoes/mosquito-control/community/what-mosquito-control-programs-do.html) which include aerial and truck spraying. A more focused method incorporating these factors will be more effective.

Other preventive measures can be introduced based on our outside research:

**Public Education**  
From our outside research, the Centers for Disease Control and Prevention (CDC) have been encouraging the general population to be vigilant and report for potential mosquito outbreaks in their area. They also have published preventive measures that the public can do to reduce the breeding of mosquitoes such as removing standing water to reduce mosquito larvae. More information are published on their [website](https://www.cdc.gov/mosquitoes/mosquito-control/community/what-mosquito-control-programs-do.html) and open for public reading which must be circulated to increase the education efforts. More can be done to help digest the information of the dangers of the West Nile Virus from their [website](https://www.cdc.gov/mosquitoes/guidelines/west-nile/introduction.html) to "bite-size" informative campaigns which can help better relay important messages to the general public.

**Regular Checks and Patrolling**  
The CDC has recommended states to employ professionals for regular checks for mosquito breeding in areas on a scheduled basis. Chicago could adopt this approach in order to combat the mosquito population and consequently reduce incidences of the virus. These professionals will be in charge of enforcing checks on households to ensure that there are no potential breeding grounds for mosquitoes by inspecting for stagnant/standing waters. At the same time, they can also educate the households during their rounds and spread the good practices to reduce potential mosquito clusters. In time, they may be able to reduce their regular checks if they see an improvement in such practices and a decrease in mosquito population in the area.

**Project Wolbachia**  

There is a growing adoption of [Project Wolbachia](https://www.worldmosquitoprogram.org/en/work/wolbachia-method/how-it-works), led by the World Mosquito Program, that helps introduce the Wolbachia bacteria that occurs naturally in 50 per cent of insect species and is safe for humans and the environment. When a species of mosquitoes called 'Aedes aegypti' carry the Wolbachia, the bacteria compete with viruses like dengue, Zika, chikungunya and yellow fever. In [Singapore](https://www.straitstimes.com/singapore/health/project-wolbachia-to-be-expanded-to-cover-about-a-third-of-hdb-blocks-from-july-grace-fu) for example,it has a positive effect in suburban areas with incidence of dengue cases reduced up to 70 per cent so far in 2022. The project already has progress in different countries like Australia and Brazil. Chicago may want to adopt this project in light of their success in different parts of the countries and being spearheaded by the World Mosquito Program whose purpose is to reduce the spread of diseases from mosquito populations. More research can be done on the effect of Wolbachia on West Nile Virus as well.